# Integrating bot with Facebook chat

## Prerequisites
1. Create a facebook page [https://www.facebook.com/pages/create/](https://www.facebook.com/pages/create/)
2. Create a chat bot attached to facebook page and get App ID and secret
3. Install facebook messenger module [https://github.com/ahmadfaizalbh/messengerbot](https://github.com/ahmadfaizalbh/messengerbot) 

##### FaceBook Messenger bot developer guide
[https://developers.facebook.com/docs/messenger-platform/complete-guide](https://developers.facebook.com/docs/messenger-platform/complete-guide)

##### Get App ID and Secret from bellow page
![Facebook app page](https://github.com/ahmadfaizalbh/Meetup-Resources/raw/master/facebook%20bot%20config%20page.png)

## webhook code

### urls.py
```python
urlpatterns = [
    ....
    url(r'^$',webhook,name="webhook"),
    ....
]
```

### views.py

#### import

```python
from django.http import HttpResponse
from django.views.decorators.csrf import csrf_exempt
from messengerbot import MessengerClient, messages, attachments, templates, elements
import json,wikipedia,urllib,os
from chatbot import Chat,reflections,multiFunctionCall
```

#### Initialization

```python
access_token = `< Access token >` 
VALIDATION_TOKEN="This_is_FFI_chatbot"
api_key = `< API key >` 
```

####  Initialize messenger client
```python
messenger = MessengerClient(access_token=access_token)
```

#### Initiate Chatbot
```python
call = multiFunctionCall({"whoIs":whoIs,
                          "whatIs":whatIs,
                          "whereIs":whereIs,
                          "tellMeAbout":tellMeAbout})


chat=Chat(os.path.join(os.path.dirname(os.path.abspath(__file__)),
                       "chatbotTemplate",
                       "Example.template"
                       ),
          reflections,call=call)
```

#### Webhook handler
```python
@csrf_exempt
def webhook(request):
    if  request.method!="POST":
        #Validate URL
        if request.GET['hub.verify_token'] == VALIDATION_TOKEN:
            return HttpResponse(request.GET['hub.challenge'])
        return HttpResponse("Failed validation. Make sure the validation tokens match.")
    return chathandler(request)    
```

#### Chat handler
```python
def chathandler(request):
    data = json.loads(request.body)
    senderID=data["entry"][0]["messaging"][0]["sender"]['id']
    recipient = messages.Recipient(recipient_id=senderID)
    # Send text message
    for i in data["entry"][0]["messaging"]:
        if "message" in i: 
            if not senderID in chat.conversation:
                #Initiate user info
                initiateChat()
            respondToClient(senderID,i["message"]["text"])
    return HttpResponse("It's working")
```

#### Initate conversation
```python
def initiateChat(senderID):
    chat._startNewSession(senderID)             
    chat.conversation[senderID].append('Say "Hello"')
    #Get Name of User from facebook
    url = "https://graph.facebook.com/v2.6/" + senderID +\
          "?fields=first_name,last_name,gender&access_token="+ access_token
    userInfo=json.load(urllib.urlopen(url))
    userInfo["name"] = userInfo["first_name"]
    chat._memory[senderID].update(userInfo)
```

#### Respond back to client by using Messengerbot
```python
def respondToClient(senderID,message):
    chat.conversation[senderID].append(message)
    while message[-1] in ".!": message = message[:-1]
    result = chat.respond(message,sessionID=senderID)               
    chat.conversation[senderID].append(result)
    response = messages.MessageRequest(recipient, messages.Message(text=result))
    #send message to Messenger
    messenger.send(response)
```